## Using K-NN Classification to Determine Subscription Status to Game Related Newsletter

#### Group 4: Skyler Lim-Voon, Evelyn Min, Lemon Campbell & Kelsey Xian

## Introduction

A research group in Computer Science at UBC, led by Frank Wood, has set up a Minecraft server to explore how people play and develop interest in video games. However, to fully understand the financial and technical needs of this project, the researchers must know which kinds of players are likely to join the server and how many resources these users will occupy. The following analysis aims to answer the research group’s broad question of “What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?” More specifically, our analysis explores if number of sessions, total number of played hours, and mean session duration can be used to predict if players will subscribe to a game-related newsletter as well as the potential variations between gender.

### Question 
#### Broad Question: 
Question 1: What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?
<br>
<br>
#### Specific Question:
Can hours played, age, and average session length predict whether a user is subscribed to a game-related newsletter? Additionally does this differ by gender?

### Data
To analyze our question, we will use the following two data sets: 
<br>
<br>


`players.csv` contains information about the users of the Minecraft server, PLAICraft. There are 196 observations and 7 variables as follows: 
- `experience` - character: player's self-determined experience level (`Beginner`, `Amateur`, `Regular`, `Veteran`, or `Pro`)
- `subscribe` - logical: player's subscription status to a game-related newsletter (`TRUE` or `FALSE`)
- `hashedEmail` - character: hashed player's email
- `played_hours` - double: total hours played by each user 
- `name` - character: player's first name
- `gender` - character: player's gender
- `Age` - double: player's age
<br>
<br>

`sessions.csv` contains information about sessions played on PLAICraft. There are 1535 observations and 5 variables as follows:
- `hashedEmail` - character: hashed player's email
- `start_time` - character: session start time in dd/mm/yyyy time 
- `end_time` - character: session end time in dd/mm/yyyy time 
- `orginal_start_time` - double: session start time in milliseconds since January 1st, 1970 at Coordinated Universal Time (UNIX time)
- `orginal_end_time` - double: session end time in milliseconds since January 1st, 1970 at Coordinated Universal Time (UNIX time)

## Methods and Results 

The code below loads the necessary libraries and data to our project.

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(lubridate)
library(ggplot2)
library(themis)
options(repr.matrix.max.rows = 10) 

players_data <- read_csv("https://raw.githubusercontent.com/skylv777/dsci100_group_project/refs/heads/main/players.csv")


sessions_data <- read_csv("https://raw.githubusercontent.com/skylv777/dsci100_group_project/refs/heads/main/sessions.csv")


 The code converts both start_time and end_time into proper datetime format so the values are tidy and creates a new column, session_length. After that, we calculated the average session length for each player and merged this summarized session-length data with the original players_data to produce a combined dataset. When we merged this data we reduced the dataset from 196 players to 192 based on the warning showing 4 removed rows in later plots. This means 4 players with no sessions are being excluded from our analysis which should not have a major impact.

In [ ]:
#Tidying sessions_data so there is only one value per cell by converting to dttm format
sessions_data_tidy <- sessions_data |>
         mutate(start_time = dmy_hm(start_time)) |>
         mutate(end_time = dmy_hm(end_time))

#Creating sessions_length column
sessions_data_difference <- sessions_data_tidy |>
        mutate(session_length = end_time - start_time) |>
        mutate(session_length = as.double(session_length))

# Determining average session length per player
average_sessions_data <- sessions_data_tidy |>
         mutate(session_length = end_time - start_time) |>
         mutate(session_length = as.double(session_length)) |>
         group_by(hashedEmail) |>
         select(session_length) |>
         summarize(average_session_length = mean(session_length))

# Combining Data Sets
combined_data <- merge(players_data, average_sessions_data)
head(combined_data)

### Summmary Statistics
​​The code computes the mean, median, mode, minimum, maximum, first quartile, third quartile, and standard deviation, while removing missing values for each quantitative variable of the combined data. A small helper function is used to calculate the mode by counting the most frequent value. 

In [ ]:
#Summary Statistics on Quantitative Values of players_data 
combined_data_select <- select(combined_data, Age, played_hours, average_session_length)

players_data_stats <- 
        bind_rows(map_df(combined_data_select, mean, na.rm = TRUE), 
                  map_df(combined_data_select, median, na.rm = TRUE),
                  map_df(combined_data_select, ~{
    x <- na.omit(.x)
    if (length(x) == 0) return(NA_real_)
    tibble(val = x) |>
      count(val, sort = TRUE) |>
      filter(n == max(n)) |>
      slice_head(n = 1) |>   
      pull(val)}),
                  map_df(combined_data_select, min, na.rm = TRUE),
                  map_df(combined_data_select, max, na.rm = TRUE), 
                  map_df(combined_data_select, sd, na.rm = TRUE), #?seems too high
                  map_df(combined_data_select, ~ quantile(.x, probs = 0.25, na.rm = TRUE)[[1]]),
                  map_df(combined_data_select, ~ quantile(.x, probs = 0.5, na.rm = TRUE)[[1]]),
                  map_df(combined_data_select, ~ quantile(.x, probs = 0.75, na.rm = TRUE)[[1]])) |>
        mutate(Summary = c("Mean", "Median", "Mode", "Minimum", "Maximum", "Standard Deviation", "1st Quartile", "2nd Quartile", "3rd Quartile")) |>
        relocate(Summary) |>
        mutate(across(Age:average_session_length, \(x) round(x, digits = 2))) 
players_data_stats

*figure 1*

### Summary Visualizations

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 15)
hours_age_subscribe_plot <- players_data |>
        ggplot(aes(x = Age, y = played_hours, colour = subscribe)) +
        geom_point() +
        labs (x = "Age",
              y = "Total hours played",
              colour = "Subscription Status to Game-Related Newsletter") +
        ggtitle("Total hours played versus Age")
hours_age_subscribe_plot   

*figure 2*

It appears that most players are under 30 and have total hours played under 10 hours especially over 30. Additionally, all the players who played more than 20 hours total were subscribed to a newsletter and users over 30 are less likely to be subscribed to a newsletter. This suggest possible correlations between subscription status, age, and total hours played. The warning indicates that two rows were removed due to missing or out-of-range values, but this does not affect the overall pattern seen in the plot.

In [ ]:
average_age_plot <- combined_data |>
        ggplot(aes(x = Age, y = average_session_length, colour = subscribe)) +
        geom_point() +
        labs (x = "Age",
              y = "Average Session Length (Minutes)",
              colour = "Subscription Status to Game-Related Newsletter") +
        ggtitle("Average Session Length versus Age")
average_age_plot  

*figure 3*

It appears that the ratio between users subscribed to users not subscribed to a newsletter is higher with players with average session length above 25 minutes. Additionally, users above the age of 30 appear more likely to have average session length below 25 minutes.

### Data Analysis

#### Creating the Model

First we set a seed, convert `subscribe` into a factor and split `combined_data` into 75% training set and 25% testing set. Then the training data is used to create a recipe to predict subscription level based on the predictors age, played hours, and average session length. The training data is also scaled and centered to ensure that the variables are equally weighted. A model is set to tune for an appropriate k value and a tibble was created for a range of k-values from 1-55. Three fold cross validation was implemented to properly fit the training data to determine the accuracy estimate of each of the potential k values. 

In [ ]:
#setting seed for reproducibility
set.seed(1)
combined_data_fct <- mutate(combined_data, subscribe = as_factor(subscribe)) |>
        drop_na()

#creating training and testing sets with 25/75 split
videogame_split <- initial_split(combined_data_fct, prop = 0.75, strata = subscribe)
videogame_train <- training(videogame_split)
videogame_test <- testing(videogame_split)  

#creating recipe including scaling and centring 
videogame_recipe <- recipe(subscribe ~ Age + played_hours + average_session_length, data = videogame_train) |>
        step_scale(all_predictors()) |>
        step_center(all_predictors())

#creating knn model 
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
        set_engine("kknn") |>
        set_mode("classification") 

k_vals <- tibble(neighbors = seq(from = 1, to = 55, by = 1))

videogame_vfold <- vfold_cv(videogame_train, v = 3, strata = subscribe)
    
videogame_wkfl <- workflow() |>
        add_recipe(videogame_recipe) |>
        add_model(knn_spec) |>
        tune_grid(resamples = videogame_vfold, grid = k_vals) |>
        collect_metrics() |> 
        filter(.metric == "accuracy") 
options(repr.plot.height = 5, repr.plot.width = 10)

k_plot <- videogame_wkfl |>
        ggplot(aes(x = neighbors, y = mean)) +
        geom_point() +
        geom_line() +
        labs(x = "Neighbors", y = "Accuracy Estimate") 
k_plot

best_k <- videogame_wkfl |>
        arrange(desc(mean)) |>
        head(1) |>
        pull(neighbors)
best_k

*figure 4*

k = 5 has the highest accuracy estimate according to the code. To confirm we can consider *figure 4* and observe that other k values have very similar accuracy estimates so considering the relatively small data set of 144 we will choose the lowest k value with that maximum accuracy estimate

#### Testing our Model
A workflow is used to fit the training data to the new model, using the original recipe and the k = 5 value we found from *figure 4*.

In [ ]:
videogame_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = best_k) |>
         set_engine("kknn") |>
         set_mode("classification") 

videogame_wkfl <- workflow() |>
        add_recipe(videogame_recipe) |>
        add_model(videogame_spec) |>
        fit(data = videogame_train) 

This model is now tested using `videogame_test` and accuracy is computed.

In [ ]:
videogame_predict <-predict(videogame_wkfl, videogame_test) |>
        bind_cols(videogame_test)

videogame_predict_accuracy <- videogame_predict |>
        metrics(truth = subscribe, estimate = .pred_class) |>
        filter(.metric == "accuracy")
conf_mat<- videogame_predict|>
    conf_mat(truth = subscribe, estimate = .pred_class)

videogame_predict_accuracy

*figure 5*

#### Majority Classifier
Determining the accuracy of the majority classifier on the entire data set for comparison with our model's accuracy

In [ ]:
videogame_proportions <- combined_data_fct |>
    group_by(subscribe) |>
    summarize(n=n()) |>
    mutate(percent = 100*n/nrow(combined_data_fct))
videogame_proportions

*figure 6*

The majority classifier has an accuracy of ~74%. This is suspeciously close to the accuracy of our model especially when we consider the class imbalance with considerably more players subscribed to a game-related newsletter in the data set. Thus, we should check the confusion matrix of our model's predictions. 

#### Confusion Matrix

In [ ]:
conf_mat<- videogame_predict|>
    conf_mat(truth = subscribe, estimate = .pred_class)
conf_mat

*figure 7*

We can observe that the model has simply predicted all `TRUE` which is problamatic as it is essentially just a majority classifier. This is likely due to the aforementioned class imbalance.

*figure 8*

#### Oversampling 
To address the issue of class imbalance we can rebalance the data by oversampling the `FALSE` subscribe class.

In [ ]:
set.seed(1)
videogame_upsample <- recipe(subscribe ~ Age + played_hours + average_session_length, data = videogame_train) |>
        step_upsample(subscribe, over_ratio = 1, skip = TRUE) |>
        prep(retain = TRUE) |>
        bake(new_data = NULL) 

upsample_vfold <- vfold_cv(videogame_upsample, v = 3, strata = subscribe)

The same tuning process done with the upsampled data.

In [ ]:
upsample_recipe <- recipe(subscribe ~ Age + played_hours + average_session_length, data = videogame_upsample) |>
        step_scale(all_predictors()) |>
        step_center(all_predictors())

upsample_wkfl <- workflow() |>
        add_recipe(upsample_recipe) |>
        add_model(knn_spec) |>
        tune_grid(resamples = upsample_vfold, grid = k_vals) |>
        collect_metrics() |> 
        filter(.metric == "accuracy") 

upsample_k_plot <- upsample_wkfl |>
        ggplot(aes(x = neighbors, y = mean)) +
        geom_point() +
        geom_line() +
        labs(x = "Neighbors", y = "Accuracy Estimate") 
upsample_k_plot

upsample_k <- upsample_wkfl |>
        arrange(desc(mean)) |>
        head(1) |>
        pull(neighbors)
upsample_k

*figure 9*

According to both the calculations done by R and *figure 8*, k = 2 yields the highest accuracy estimate. 

#### Predictions 
A workflow is used to fit the training data to the new model, using the original recipe and the k = 2 value we found from *figure 8*.This model is then tested using `videogame_test` and accuracy is computed.

In [ ]:
upsample_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = upsample_k) |>
         set_engine("kknn") |>
         set_mode("classification") 

upsample_wkfl <- workflow() |>
        add_recipe(upsample_recipe) |>
        add_model(upsample_spec) |>
        fit(data = videogame_upsample) 

upsample_predict <-predict(upsample_wkfl, videogame_test) |>
        bind_cols(videogame_test)

upsample_predict_accuracy <- upsample_predict |>
        metrics(truth = subscribe, estimate = .pred_class) |>
        filter(.metric == "accuracy")
upsample_conf_mat<- upsample_predict|>
    conf_mat(truth = subscribe, estimate = .pred_class)

upsample_predict_accuracy

*figure 10*

The accuracy here is 

In [ ]:
upsample_conf_mat

*figure 11*

*figure 12*

#### Analysis by Gender

In [ ]:
options(repr.plot.height = 7, repr.plot.width = 10)
gender_bar <- players_data |>
        ggplot(aes(x = gender)) +
        geom_bar(aes(fill = subscribe)) +
        ggtitle("Subscription Status by Male and Female") +
        labs(x = "Gender",
             y = "Ratio between subscription status",
             fill = "Subscription Status")
gender_bar

*figure 13*

The bar chart compares subscription status across gender categories. Male players dominate the largest portion of the data set overall, and thus they have the highest number of both subscribers and non subscribers. The second largest gender category is Female with again a majority subscribed. Due to the minimal number of players in other gender groups, it is challenging to make comparable statements. Overall, the plot suggests the subscription patterns mostly reflect the unequal distribution of genders in the dataset. 


In [ ]:
options(repr.plot.height = 5, repr.plot.width = 10)
gender_ratio <- players_data |>
        ggplot(aes(x = gender)) +
        geom_bar(aes(fill = subscribe), position = "fill") +
        ggtitle("Ratio of Subscription Status by Gender") +
        labs(x = "Gender (M or F)",
             y = "Amount",
             fill = "Subscription Status")
gender_ratio    


*figure 14*

The plot shows the proportion of subscribers and non-subscribers within each gender category using a fill-ratio bar chart. Most genders display a similar pattern, where a larger proportion of individuals are subscribed (True) compared to not subscribed (False). A few groups with smaller sample sizes show more variation, but this likely reflects limited data (*figure 11*) rather than a real difference. Overall, the chart suggests that subscription ratios are fairly consistent across genders. 

## Discussion 

#### Summary
To answer our specific question of "Can hours played, age, and average session length predict whether a user is subscribed to a game-related newsletter?" we utilized KNN classification. We examined potential variations in subscription status through visualizations.
<br>
<br>
Using our model that was tuned off the training data, we found an accuracy of 74.19% when used to predict on the testing data. However, this was due to the model predicting only `True` values and the class imbalance between the subscribed and not subscribed. Therefore this model is not sufficient to predict user's subscription status.
<br>
<br>
In an effort to fix this we balanced the data. Unfortunately, this model is also not sufficient to predict user's subscription status. The model tuned with the balanced training data was 58% accurate on the testing data which is worse than the 73.55% accuracy found of the majority classifier. It incorrectly classified 46.67% of the subscribed players as not subscribed. However, it accurately predicted which users were not subscribed. This indicates that the model trained with the oversampled data is more useful than our previous model.
<br>
<br>
Overall, through our models we were unable to predict subscription status based on different variables for the users in the dataset. 
<br>
<br>
In relation to gender, there is a very unequal distribution of genders as the users are male dominated. The visuals created led to no clear evidence that gender has any effect on subscription status. The differences in *figure 12* are most likely due to lack of equal gender sampling. 
<br>
<br>
  *and found unclear results in relation to the effect of gender on subscription. This information gives more insight into the characteristics and behaviours that are most predictive of subscribing to a game-related newsletter.* In addition, the visualizations and analysis allowed for relationships between the variables to be determined and to better understand how they differ based on the player.

#### Expectations vs Findings
Before building our classifier, we expected the three behavioural variables—total hours played, age, and average session length—to contribute differently to predicting subscription status. Our exploratory plots suggested that players with higher total playtime and longer average sessions were slightly more likely to be subscribed, so we anticipated these two engagement-related variables would provide stronger predictive value. In contrast, we expected age to have a weaker effect, since most players across age groups accumulated relatively low total hours.
For the secondary question of whether these patterns differ by gender, we expected any differences to be small, and we were unsure whether the heavily male-dominated dataset would allow for a meaningful comparison.
After training and tuning the K-NN classifier, our findings largely aligned with these expectations. The model achieved an accuracy of 74.19%, and the playtime-related variables appeared more informative than age, which showed weaker and less consistent trends in the visualizations. Regarding gender, we found no meaningful difference in subscription patterns, with similar subscriber proportions for male and female players. Overall, the behavioural variables produced a reasonably accurate model, while gender did not appear to influence subscription outcomes in this dataset.

#### Potential Impacts
If we assume that players who are subscribed to game-related newletter are generally more interesting in recieving information about video games. Therefore, if a company can predict subscription, they can also build an email list that target those most likely to participate in competitions or to make game-related purchases to improve monetization and engagment. Additionally, predicting receptivity to newsletter can increase brand trust as recieving irrelevant emails can irritate customers. 

#### Future Questions

- Does session length change before or after a player subscribes to a newsletter?
- Is there an age group that is more likely to subscribe than others?
- How can we make this classifier more generalizable for predicting subscription behaviour in other gaming communities or servers?
- What additional behavioural features (e.g., time of day played, frequency of logins) could improve the accuracy of the classifier?
- Would adding more behavioural features improve the model’s accuracy?
- Would a larger training set improve the model's accuracy?


## GitHub Link